In [1]:
import glob
files = glob.glob(r'D:\Google雲端硬碟\NCKU\法經\法經報告\procedure\判決書\*\*.txt')

In [8]:
# 將中文數字變成阿拉伯數字的function
capital_chars = {
    u'零': "0",
    u'壹': "1",
    u'貳': "2",
    u'參': "3",
    u'': "3",
    u'': "3",
    u'肆': "4",
    u'伍': "5",
    u'陸': "6",
    u'柒': "7",
    u'捌': "8",
    u'玖': "9",
}

simplified_chars = {
    u"零" : "0",
    u"一" : "1",
    u"二" : "2",
    u"三" : "3",
    u"四" : "4",
    u"五" : "5",
    u"六" : "6",
    u"七" : "7",
    u"八" : "8",
    u"九" : "9",
}

def to_decimal(list_to_convert):
    result = ''
    list_converted = []
    for element in list_to_convert:
        for char in element:
        
            if char in capital_chars: # 如果遇到大寫數字，先存起來
                result += capital_chars[char]

            elif char in simplified_chars: # 如果遇到簡體數字，先存起來
                result += simplified_chars[char]

            elif char == "點":
                result += "."

            else:
                result += char
        list_converted.append(result)
        result = ""
    return list_converted

In [9]:
def main_part_BAL(main_part): # 捕捉BAL的function, 呼氣濃度
    import re
    bal = []
    bal_context = ["每公升","mg/L"]
    bal_conv = []
    bal_float = []
    real_bal = 0

    main_part = main_part.strip().replace("\n","").replace("\u3000","").replace(" ","") # 對文檔做處理
    txt_list = main_part.split("書記官") # 以"書記官"split 

    for i in bal_context: # 以context判斷BAL在哪裏
        iteration = re.finditer(i, txt_list[0])
        indices = [m.start(0) for m in iteration]
        for index in indices:
            if i == "每公升":
                bal.append(main_part[index + 3 : index + 7])
            elif i == "mg/L":
                bal.append(main_part[index - 4 : index])
            elif i == "mg/l":
                bal.append(main_part[index - 4 : index])

        bal_conv = to_decimal(bal) # symbol conversion

        while "0.25" in bal_conv: # drop 0.25 in excess
            bal_conv.remove("0.25")

        while "0.05" in bal_conv: # drop 0.05, 這是血液濃度
            bal_conv.remove("0.05")

        if not bal_conv: # append 0.25 as the lowest boundary，有些的值剛剛好落在0.25，所以在前面drop 0.25會被去除，補一個0.25，作爲最低標準
            bal_conv.append("0.25")

        try:
            for element in bal_conv: # str to float
                bal_float.append(float(element)) 
                real_bal = max(bal_float) # max value
                return real_bal   
        except:
            return "Error"

In [ ]:
o = []
for i,file in enumerate(files):
#file = files[0]
    if i % 2700 == 0:
        print(i)
    with open(file,'r',encoding='utf-8-sig') as f:
        txt =f.read()
    hol = main_part_BAL(txt)
    o.append(hol)

# main

In [ ]:
import glob
files = glob.glob(r'D:\Google雲端硬碟\NCKU\法經\法經報告\procedure\判決書\*\*.txt')

In [136]:
def set_part(file):
    with open(file,'r',encoding='utf-8-sig') as f:
        txt =f.read()
    part = txt.replace("\n","").replace("\u3000","").replace(" ","").replace('點','.')
    part = part.replace('．','.').replace('‧','.').replace('│','')
    # 改成阿拉伯數字/
    for ele in tran_num:
        part = part.replace(ele ,tran_num[ele])
    # /改成阿拉伯數字
    return part

In [218]:
import re
file_paths = []
file_names=[]
alos=[]
other=[]
tran_num = {u"零" : "0",u"一" : "1",u"二" : "2",u"三" : "3",u"四" : "4",u"五" : "5",u"六" : "6",u"七" : "7",u"八" : "8",
            u"九" : "9",'十':'','０':'0' ,'１':'1','２':'2','３':'3','４':'4','５':'5','６':'6','７':'7','８':'8','９':'9',
            u'壹': "1",u'貳': "2",u'參': "3",u'': "3",u'': "3",u'肆': "4",u'伍': "5",u'陸': "6",u'柒': "7",u'捌': "8",
            u'玖': "9",'○':'0'}

for i,file in enumerate(files):
    #file = files[0]
    if i % 2700 == 0:
        print(i)
    part = set_part(file)
    input_text = part
    regex = '氣.{1,20}毫克'
    pattern = re.compile(regex)
    match = re.findall(pattern, input_text)
    max_ = 0.0
    if match != []:
        for text in match:
            if '升.' in text: #少打0
                regex = '.\d'
                pattern = re.compile(regex)
                num = re.search(pattern, text).group()
                num = '0'+num
            elif '.' not in text: #整數時
                regex = '\d'
                pattern = re.compile(regex)
                if re.search(pattern, text) == None: #確定該text無數字
                    continue
                num = re.search(pattern, text).group()
            else:
                regex = '\d.\d+'
                pattern = re.compile(regex)
                num = re.search(pattern, text).group()
            if float(num) > float(max_):
                max_ = num
        if max_ == '0.25': #可能是只有測血液沒測呼氣
            regex = '血.{1,30}(\d\d\d\.\d+)|(\d\.\d+)' #因有兩種表達方式
            pattern = re.compile(regex)
            match2 = re.findall(pattern, part)  #這裡用part
            nums = []
            for b in match2: #將兩種化成同單位
                if b[1] != ''and b[1] != '0.25' and float(b[1]) <0.9:
                    nums.append( float(b[1])*5 )
                if b[0] != '':
                    nums.append( float(b[0])*0.005 )
            for n in nums:
                if n > float(max_):
                    max_ = n
        file_paths.append(file)
        file_name = file.split('\\')[-1]
        file_names.append(file_name)
        alos.append( float(max_) )
    else:
        other.append([i,file])
#最後alcohol=0.25的有2142件，除了沒寫以外，可能還存在誤判的可能。

0
2700
5400
8100
10800
13500
16200
18900
21600
24300


In [222]:
len(alos),len(other),len(set(alos))

(26875, 10, 721)

# save

In [224]:
import pandas as pd
df = pd.DataFrame(data={'index':file_names,'alcohol': alos})
path = 'D:/Google雲端硬碟/NCKU/法經/法經報告/procedure/csv/index_alcohol.csv'
df.to_csv(path,encoding='utf-8-sig',index=False)
data = pd.read_csv('D:/Google雲端硬碟/NCKU/法經/法經報告/procedure/csv/data26885_4.csv',encoding='ansi')
data.shape

(26885, 128)

In [226]:
df.set_index(['index'],inplace = True)
data.set_index(['index'],inplace = True)
data2 = pd.concat([data,df],axis='columns')
data2.reset_index(inplace=True)
data2

,court,date,case_kind,case_num,judge,jail,week,laws,'crime_1','crime_11',...,quaotou,shilin,taichung,tainan,taipei,taitung,taoyuan,yilan,yunlin,alcohol
南投_2019-01-02_投交簡_576.txt,nantou,2019/1/2,投交簡,576,孫于淦,2,1,"'suit_449', 'suit_454', 'crime_185-3', 'crime_...",0,0,...,0,0,0,0,0,0,0,0,0,1.075
南投_2019-01-07_埔交簡_2.txt,nantou,2019/1/7,埔交簡,2,孫于淦,3,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_41'",0,0,...,0,0,0,0,0,0,0,0,0,0.540
南投_2019-01-07_投交簡_521.txt,nantou,2019/1/7,投交簡,521,楊國煜,2,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_41'",0,0,...,0,0,0,0,0,0,0,0,0,0.290
南投_2019-01-07_投交簡_524.txt,nantou,2019/1/7,投交簡,524,孫于淦,5,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_41'",0,0,...,0,0,0,0,0,0,0,0,0,0.250
南投_2019-01-07_投交簡_531.txt,nantou,2019/1/7,投交簡,531,黃俊明,5,2,"'suit_449', 'suit_454', 'crime_185-3', 'crime_...",0,0,...,0,0,0,0,0,0,0,0,0,0.430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
高雄_2019-06-18_審交易_437.txt,kaohsiung,2019/6/18,審交易,437,詹尚晃,8,25,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,0,...,0,0,0,0,0,0,0,0,0,0.700
高雄_2019-07-16_審交易_528.txt,kaohsiung,2019/7/16,審交易,528,詹尚晃,16,29,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,0,...,0,0,0,0,0,0,0,0,0,0.660
高雄_2019-08-15_審交易_625.txt,kaohsiung,2019/8/15,審交易,625,詹尚晃,10,33,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,0,...,0,0,0,0,0,0,0,0,0,0.500
高雄_2019-08-20_審交易_652.txt,kaohsiung,2019/8/20,審交易,652,詹尚晃,5,34,"'suit_273-1', 'suit_299', 'crime_185-3', 'crim...",0,0,...,0,0,0,0,0,0,0,0,0,0.590


In [227]:
data2.to_csv('D:/Google雲端硬碟/NCKU/法經/法經報告/procedure/csv/data26885_5.csv',encoding='utf-8-sig',index=False)
data2.to_csv('D:/Google雲端硬碟/NCKU/法經/備份檔案/data26885_5.csv',encoding='utf-8-sig',index=False)

# test---------------------------------------------------------------------

In [221]:
count = 0
for i,n in enumerate(alos):
    if n == 0.25:
        count = count +1
        #print(i,file_names[i],end=' \n')    
count

2142

In [199]:
target = '0.25'
count = 0
import re
file_names=[]
alos=[]
other=[]
thing = []
tran_num = {u"零" : "0",u"一" : "1",u"二" : "2",u"三" : "3",u"四" : "4",u"五" : "5",u"六" : "6",u"七" : "7",u"八" : "8",
            u"九" : "9",'十':'','０':'0' ,'１':'1','２':'2','３':'3','４':'4','５':'5','６':'6','７':'7','８':'8','９':'9',
            u'壹': "1",u'貳': "2",u'參': "3",u'': "3",u'': "3",u'肆': "4",u'伍': "5",u'陸': "6",u'柒': "7",u'捌': "8",
            u'玖': "9",'○':'0'}

for i,file in enumerate(files):
    #file = files[0]
    if i % 2700 == 0:
        print(i)
    part = set_part(file)
    input_text = part
    regex = '氣.{1,20}毫克'
    pattern = re.compile(regex)
    match = re.findall(pattern, input_text)
    max_ = 0.0
    if match != []:
        for text in match:
            if '升.' in text: #少打0
                regex = '.\d'
                pattern = re.compile(regex)
                num = re.search(pattern, text).group()
                num = '0'+num
            elif '.' not in text: #整數時
                regex = '\d'
                pattern = re.compile(regex)
                if re.search(pattern, text) == None: #確定該text無數字
                    continue
                num = re.search(pattern, text).group()
            else:
                regex = '\d.\d+'
                pattern = re.compile(regex)
                num = re.search(pattern, text).group()
            if float(num) > float(max_):
                max_ = num
        file_name = file.split('\\')[-1]
        file_names.append(file_name)
        alos.append(max_)
        if max_ =='0.25':
            count = count +1
            print(i,file,'\n',part)
            if count == 10:
                break
    else:
        other.append([i,file])


0
0 D:\Google雲端硬碟\NCKU\法經\法經報告\procedure\判決書\南投\南投_2019-01-02_投交簡_576.txt 
 臺灣南投地方法院刑事簡易判決107年度投交簡字第576號聲請人臺灣南投地方檢察署檢察官被告陳彥廷上列被告因公共危險案件，經檢察官聲請以簡易判決處刑（107年度偵字第5342號），本院判決如下：主文陳彥廷駕駛動力交通工具而其血液中酒精濃度達百分之0.05以上，累犯，處有期徒刑5月，如易科罰金，以新臺幣1仟元折算1日。犯罪事實及理由1、犯罪事實：陳彥廷於民國107年9月15日20、21時許，在其位於南投縣水里鄉友人住處飲用啤酒，明知飲用酒類後不能駕駛動力交通工具，猶基於酒後駕駛動力交通工具之犯意，於飲畢後之隔日凌晨某時許無照騎乘車牌號碼000-000號普通重型機車，自上揭飲酒處出發而行駛於道路上。嗣於同年9月16日0時55分許，行經南投縣00鄉00路000號前，因酒後無法安全操控其所駕駛之車輛，不慎自行摔倒而受傷並送醫。嗣經警據報前往處理，並委由竹山秀傳醫院對陳彥廷實施血液中酒精濃度測試後，於同年9月16日2時14分測得其血液中之酒精濃度值為215mg/dL（即百分之0.215），已達百分0.05以上，始查悉上情。2、認定犯罪事實所憑之證據及理由：上揭被告飲酒後騎乘機車肇事受傷送醫，嗣為警委託醫院對其抽血檢驗，測得其血液中酒精濃度值達215MG/DL（即百分之0.215），超過法規規定標準等事實，業據被告於警、偵詢時所坦承，復有南投縣政府警察局集集分局交通事故處理小組道路交通事故當事人酒精測定紀錄表（其上黏貼竹山秀傳醫院生化檢查報告）、道路交通事故現場圖、道路交通事故調查表（1）、道路交通事故調查報告表（2）、南投縣000000000道路00000000000號查詢機車駕駛人、車號查詢機車車籍等件及道路交通事故照片12張在卷可稽。足認被告之自白與事實相符，本件被告經測得血液中酒精濃度值為百分之0.215，已超過百分之0.05以上之標準，是被告上開罪行，應堪予認定。3、論罪科刑之理由：（1）按刑法第185條之3第1項第1款後段之血液中酒精濃度達百分之0.05以上之要件，係以酒精濃度標準值，作為認定不能安全駕駛之判斷標準，即被告飲酒後駕駛動力交通工具，經查獲後其血液中之酒精濃度超過法規標準即堪認符合犯罪構成要件。至於其主觀犯

In [193]:
file =files[49]
part = set_part(file)
                     
input_text = part
regex = '氣.{1,20}毫克'
pattern = re.compile(regex)
match = re.findall(pattern, input_text)
#print(match,part)
max_ = 0.0
if match != []:
    for text in match:
        if '升.' in text: #少打0
            regex = '.\d'
            pattern = re.compile(regex)
            num = re.search(pattern, text).group()
            num = '0'+num
        elif '.' not in text: #整數時
            regex = '\d'
            pattern = re.compile(regex)
            if re.search(pattern, text) == None: #確定該text無數字
                continue
            num = re.search(pattern, text).group()
        else:
            regex = '\d.\d+'
            pattern = re.compile(regex)
            num = re.search(pattern, text).group()
        if float(num) > float(max_):
            max_ = num
    if max_ == '0.25': #可能是只有測血液沒測呼氣
        regex = '血.{1,30}(\d+\.\d+)|血.{1,30}(\d+)' #因有兩種表達方式
        pattern = re.compile(regex)
        match2 = re.findall(pattern, part)  #這裡用part
        nums = []
        for b in match2: #將兩種化成同單位
            if b[0] != '':
                nums.append( float(b[0])*5 )
            if b[1] != '':
                nums.append( float(b[1])*0.005 )
        for n in nums:
            if n > float(max_):
                max_ = n
max_

1.1265

In [189]:
test ='''學成駕駛動力交通工具而血液中酒精濃度達百分之0.05以上，處有期徒刑3月，如易科罰金，以新臺幣1仟元折算1日。事實及理由1、本案犯罪事實
及證據，除犯罪事實部分關於「血液中酒精濃度為百分之0.2495」之記載應更正為「血液中酒精濃度為百分之0.2498」'''
regex = '血.{1,30}(\d+\.\d+)|血.{1,30}(\d+)' #因有兩種表達方式
pattern = re.compile(regex)
match2 = re.findall(pattern, test)
nums = []
for b in match2: #將兩種化成同單位
    if b[0] != '':
        nums.append( float(b[0])*5 )
    if b[1] != '':
        nums.append( float(b[1])*0.005 )
for n in nums:
    if n > float(max_):
        max_ = n
max_

1.2489999999999999

In [163]:
for i,n in enumerate(other):
    file = n[1]
    part = set_part(file)
    print(file,part)

D:\Google雲端硬碟\NCKU\法經\法經報告\procedure\判決書\南投\南投_2019-02-23_埔交簡_194.txt 臺灣南投地方法院刑事簡易判決107年度埔交簡字第194號聲請人臺灣南投地方檢察署檢察官被告田進權上列被告因公共危險案件，經檢察官聲請以簡易判決處刑（107年度偵字第5106號），本院判決如下：主文田進權駕駛動力交通工具而血液中酒精濃度達百分之0.05以上，處有期徒刑2月，如易科罰金，以新臺幣1仟元折算1日。事實及理由1、本案犯罪事實及證據，除應補充、更正如下外，餘均引用檢察官聲請簡易判決處刑書之記載（如附件）：犯罪事實：自撞所受傷害之情形為「未明示側性前胸壁挫傷、頭部未明示部位鈍傷、下背和骨盆挫傷」。證據：「道路交通事故調查報告表、」之記載應更正並補充為「道路交通事故調查表、道路交通事故調查報告表」。2、核被告田進權所為，係犯刑法第185條之3第1項第1款駕駛動力交通工具而血液中酒精濃度達百分之0.05以上之罪。3、本院審酌被告：1.未有何酒後駕車前科紀錄，有臺灣高等法院被告前案紀錄表1份在卷可稽，此次為初犯；2.其明知酒精對人之意識能力具有不良影響，酒後行車對1般往來之公眾及駕駛人自身皆具有高度危險性，竟於血液中酒精濃度達百分之0.3571，已達百分之0.05以上甚鉅，仍駕駛自用小客車搭載友人上路，漠視公權力及往來人車生命、身體、財產安全，對交通安全危害非輕；3.其因此不慎自撞路邊電線桿，致己及友人受有上述之傷害，並因而為警查獲，惟幸未造成其他無辜用路人之傷亡；4.兼衡其高職畢業之智識程度、家庭經濟狀況勉持；5.暨犯後坦承犯行之態度等1切情狀，量處如主文所示之刑，並諭知易科罰金之折算標準。4、適用之法律：刑事訴訟法第449條第1項前段、第454條第2項。5、如不服本判決，應於送達後10日內，以書狀敘明理由，向本院提出上訴。本案經檢察官洪紹文聲請以簡易判決處刑。中華民國108年2月23日臺灣南投地方法院埔里簡易庭法官蔡如惠以上正本證明與原本無異。如不服本判決應於判決送達後日內向本院提出上訴狀，（均須按他造當事人之數附繕本）「切勿逕送上級法院」。書記官吳瓊英中華民國108年2月25日
